In [55]:
# Libraries

import numpy as np 
import matplotlib.pyplot as plt
from scipy.integrate import quad
import ipywidgets as widgets
import warnings

In [56]:
# This avoids warning messages caused by large exponential values
warnings.filterwarnings("ignore", category=RuntimeWarning)
np.seterr(over='ignore')

{'divide': 'warn', 'over': 'ignore', 'under': 'ignore', 'invalid': 'warn'}

In [57]:
# Constants CGS
c = 2.9979e10 # cm s^-1
kb = 1.3806e-16 # erg K^-1
h = 6.626e-27 # erg s^-1
sigma = 5.6704e-5 # erg s^-1 cm^-2 K^-4

In [58]:
def planck_law(T):

    """
    Calculates the Planck distribution function, describing the radiation emitted
    by a black body as a function of the wavelength and temperature, expressed in nanometers.

    PARAMETERS:
        
       T (float): Temperature of the black body in Kelvin
    
    RETURN:
        tuple:

        - wavelength (np.ndarray): Array of wavelenghts in nanometers.
        - planck_function_nm (np.ndarray): Spectral energy density (erg·s⁻¹·cm⁻²·nm⁻¹) for
        each wavelength

    Note:

        The internal function is calculated in CGS units and then is converted to nanometers multiplying by 1e-7.
    """

    wavelength = np.linspace(1, 4000, 1000) #nm
    wavelength_cm = wavelength*1e-7

    planck_function = ((2*h*(c**2)) / (wavelength_cm**5)) * (1 / (np.exp((h*c)/(wavelength_cm*kb*T)) - 1))
    planck_function_nm = planck_function*1e-7

    return (wavelength, planck_function_nm)

In [59]:
def wien_law(T):

    """
    Calculates the wavelength at which a black body emits maximun radiation at a given
    temperature, using Wien's displacement law.

    PARAMETERS:

        T (float): Temperature of the black body in Kelvin
    
    RETURN:

        float: wavelength (in nanometers) corresponding to the peak of emission.

    Note:

        The Wien's constant used is 0.2898 cm·K. The result is converted from cm to nm.
    """

    lambda_max = 0.2898/T # cm
    return lambda_max*1e7 # nm y K

In [60]:
def energy(T):

    """ 
    Calculates the total energy emitted per unit of area, per unit of time and per unit of surface
    (energetic flux) by a black body at temperature T, by integrating the Planck's law expressed in 
    terms of an adimensional variable over the entire wavelength range.

    PARAMETERS:

        T (float): Temperature of the black body in Kelvin

    RETURN:

        float: Total energy integrated over all wavelengths, in erg·s⁻¹·cm⁻².

    Notes:

        - The integration uses the adimensional form of Planck's law, where the integration 
          variable x = (h*c) / (λ*k_B*T).
        - The integral is performed from 0 to infinity in terms of x.
        - The constant factor outside the integral converts the adimensional integral back to 
        physical units, assuming CGS units for constants.

    """
    constant = (2*((kb*T)**4)) / ((h**3)*(c**2))
    
    def adimensional_function(x, T):

        adimensional_function = (x**3) / (np.exp(x) - 1)
        return adimensional_function

    energy_planck, error_planck = quad(adimensional_function, 0, np.inf, args=(T,)) #erg cm^−2 s^−1
    energy_planck = np.pi*constant*energy_planck

    return energy_planck

In [61]:
def percentage_energy(T, lmbda1, lmbda2):

    """
    Calculates the percentage of the total flux emitted by a black body at temperature T 
    that lies within a specific wavelength range, using the adimensional form of Planck's law.

    PARAMETERS:

        - T (float): Temperature of the black body in Kelvin.
        - lmbda1 (float): Lower limit of the wavelength in nanometers.
        - lmbda2 (float): Upper limit of the wavelength in nanometers.

    RETURN:

        float: Percentage of total emitted flux that lies between lmbda1 and lmbda2 (in %).

    Notes:

        - The Planck function is expressed in terms of the adimensional variable 
          x = (h*c)/(λ*k_B*T), and the integration is performed over x1 to x2.
        - The physical wavelength limits are converted to adimensional x-limits 
          before integration.
        - The result is normalized with respect to the total flux computed from 
          the full integral (0 to infinity) in adimensional form.
        - All constants are in CGS units, and the final result is a dimensionless 
          percentage. 

    """

    lmbda1_cm, lmbda2_cm = lmbda1*1e-7, lmbda2*1e-7

    constant = (2*((kb*T)**4)) / ((h**3)*(c**2))
    
    def adimensional_function(x, T):

        adimensional_function = (x**3) / (np.exp(x) - 1)
        return adimensional_function
    
    x1 = (h*c) / (lmbda2_cm * kb * T) 
    x2 = (h*c) / (lmbda1_cm * kb * T)

    energy_planck_range, error_planck_range = quad(adimensional_function, x1, x2, args=(T,)) #erg cm^−2 s^−1
    
    energy_fraction = (np.pi*constant*energy_planck_range / energy(T)) * 100
    
    return round(energy_fraction, 4)

In [62]:
def plot_by_temperature(T):

    """ 
    Plots the Planck's law at a given temperature and displays key information
    about the black body spectrum corresponding to a star.

    PARAMETERS:

        - T (float): Temperature of the black body (star) in Kelvin.

    Notes:

        The function generates a plot of the spectral flux according to the Planck's law as
        a function of wavelength (in nanometers) for a giventemperature T. Additionally:

            - Shows the peak wavelength obtained using Wien's law
            - Calculates and displays the total total emitted energy flux.
            - Classifies the star approximately according to its spectral type (O, B, A, F, G, K, M).

        The curve is expressed in CGS units:
            - Wavelength in nanometers (nm)
            - Flux in erg s⁻¹ cm⁻² cm⁻¹.
    """

    if T >= 30000:
        type_star = "O"
    elif 30000 > T and T >= 10000:
        type_star = "B"
    elif 10000 > T and T >= 7500:
        type_star = "A"
    elif 7500 > T and T >= 6000:
        type_star = "F"
    elif 6000 > T and T >= 5000:
        type_star = "G"
    elif 5000 > T and T >= 3500:
        type_star = "K"
    elif 3500 > T and T >= 2500:
        type_star = "M"
    else:
        type_star = "Unclassified"

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(planck_law(T)[0], planck_law(T)[1] , color = "#005f99", label = f"Temperature: {T} (K)")
    ax.axvline(wien_law(T), ls = "--", color = "#005f99", label = f"Max wavelength: {wien_law(T):.3e} (nm)")
    ax.plot([], [], ' ', label = rf"Energy: {energy(T):.4e} (erg cm$^{{-2}}$ s$^{{-1}})$")
    ax.plot([], [], ' ', label = f"Type star: {type_star}")

    ax.set_xlabel(r"$\mathrm{Wavelength} \ (\mathrm{nm})$")
    ax.set_ylabel(r"$F_\lambda \ (\mathrm{erg\,s^{-1}\,cm^{-2}\,cm^{-1}})$")
    ax.grid()
    ax.legend(fontsize = 12)
    plt.show()

In [63]:
def plot_energy_in_a_range(T, lmbda1, lmbda2):

    """ 
    Plots the black body spectrum  for a given temperature T, highlighting the wavelength range
    of interest and displaying the fraction of energy emitted in that range.

    PARAMETERS:

        - T (float): Temperature of the black body in Kelvin.
        - lmbda1 (float): Lower limit of the spectrum range (in nanometers).
        - lmbda2 (float): Upper limit of the spectrum range (in nanometers).

    Notes:

        This function generates a plot of the spectral flux according to the Planck's law for a
        given temperature T, highlighting the region between lmbda1 and lmbda2 with vertical lines.

    Additionally:

        - Displays the fraction of the total emitted flux within the defined range.
        - Classifies the star approximately according its spectral type (O, B, A, F, G, K, M).
        - Shades characteristic spectral bands:
            Ultraviolet (UV), visible, Near infrared (NIR) and Mid infrared (MIR).

    Units follows the CGS system:
    
        - Wavelength: nanometers (nm).
        - Spectral flux: erg s⁻¹ cm⁻² cm⁻¹.
    """

    if T >= 30000:
        type_star = "O"
    elif 30000 > T and T >= 10000:
        type_star = "B"
    elif 10000 > T and T >= 7500:
        type_star = "A"
    elif 7500 > T and T >= 6000:
        type_star = "F"
    elif 6000 > T and T >= 5000:
        type_star = "G"
    elif 5000 > T and T >= 3500:
        type_star = "K"
    elif 3500 > T and T >= 2500:
        type_star = "M"
    else:
        type_star = "Unclassified"

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.plot(planck_law(T)[0], planck_law(T)[1] , color = "#005f99", label = f"Temperature: {T} (K)")
    ax.plot([], [], ' ', label=f"Fraction of Energy: {percentage_energy(T, lmbda1, lmbda2)} %")
    ax.plot([], [], ' ', label=f"Type star: {type_star}")

    ax.axvspan(10, 400, color='purple', alpha = 0.13)
    ax.axvspan(400, 780, color='green', alpha = 0.13)
    ax.axvspan(780, 2500, color='red', alpha = 0.13)
    ax.axvspan(2500, 4000, color='red', alpha = 0.2)

    ax.axvline(lmbda1, ls = "--", color = "gray")
    ax.axvline(lmbda2, ls = "--", color = "gray")

    ax.text(205, 0.45e6, "UV", ha='center', va='bottom', fontsize=15, alpha=0.8)
    ax.text(590, 0.45e6, "Visible", ha='center', va='bottom', fontsize=15, alpha=0.8)
    ax.text(1640, 0.45e6, "NIR", ha='center', va='bottom', fontsize=15, alpha=0.8)
    ax.text(3250, 0.45e6, "MIR", ha='center', va='bottom', fontsize=15, alpha=0.8)

    ax.set_xlabel(r"$\mathrm{Wavelength} \ (\mathrm{nm})$")
    ax.set_ylabel(r"$F_\lambda \ (\mathrm{erg\,s^{-1}\,cm^{-2}\,cm^{-1}})$")
    ax.grid()
    ax.legend(fontsize = 12)
    plt.show()  

In [64]:
widgets.interact(plot_by_temperature, T = widgets.FloatSlider(value=3000, min=500, max= 35000, step=1, description = "T (K)", layout={'width': '600px'}))

interactive(children=(FloatSlider(value=3000.0, description='T (K)', layout=Layout(width='600px'), max=35000.0…

<function __main__.plot_by_temperature(T)>

In [65]:
widgets.interact(plot_energy_in_a_range, T = widgets.FloatSlider(value=3000, min=500, max= 35000, step=1, description = "T (K)", layout={'width': '600px'}), 
         lmbda1 = widgets.FloatText(value=400, description='λ₁ (nm)'), 
         lmbda2 = widgets.FloatText(value=700, description='λ₂ (nm)'))

interactive(children=(FloatSlider(value=3000.0, description='T (K)', layout=Layout(width='600px'), max=35000.0…

<function __main__.plot_energy_in_a_range(T, lmbda1, lmbda2)>

In [66]:
help(planck_law)

Help on function planck_law in module __main__:

planck_law(T)
    Calculates the Planck distribution function, describing the radiation emitted
    by a black body as a function of the wavelength and temperature, expressed in nanometers.
    
    PARAMETERS:
        
       T (float): Temperature of the black body in Kelvin
    
    RETURN:
        tuple:
    
        - wavelength (np.ndarray): Array of wavelenghts in nanometers.
        - planck_function_nm (np.ndarray): Spectral energy density (erg·s⁻¹·cm⁻²·nm⁻¹) for
        each wavelength
    
    Note:
    
        The internal function is calculated in CGS units and then is converted to nanometers multiplying by 1e-7.

